In [1]:
!pip install pytorch-optimizer
!pip install fastfeedforward
!pip install triton
!pip install liger-kernel
!pip install jiwer
!pip install huggingface_hub safetensors

In [2]:
from huggingface_hub import login, HfApi, HfFolder
from safetensors.torch import save_file

# Log in to your Hugging Face account
login() # Follow the instructions to get your token

In [3]:
# For demonstration, we use synthetic dataset

import requests
import json

# URL containing the JSON data
url = 'https://huggingface.co/datasets/ChavyvAkvar/synthetic-number/resolve/main/corpus_texts_train.json?download=true'

# Send a GET request to the URL
response = requests.get(url)

texts = []

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON data
    texts = response.json()

    # Check if the data is a list
    if isinstance(texts, list):
        # Now `data` is a Python list (array)
        print(texts[5])
    else:
        print("The data is not an array.")
else:
    print(f"Failed to retrieve data: {response.status_code}")

print(len(texts))

seventy nine thousand one hundred and thirty nine seventy nine thousand one hundred and forty seventy nine thousand one hundred and forty one seventy nine thousand one hundred and forty two seventy nine thousand one hundred and forty three seventy nine thousand one hundred and forty four seventy nine thousand one hundred and forty five seventy nine thousand one hundred and forty six seventy nine thousand one hundred and forty seven seventy nine thousand one hundred and forty eight seventy nine thousand one hundred and forty nine seventy nine thousand one hundred and fifty seventy nine thousand one hundred and fifty one seventy nine thousand one hundred and fifty two seventy nine thousand one hundred and fifty three seventy nine thousand one hundred and fifty four seventy nine thousand one hundred and fifty five seventy nine thousand one hundred and fifty six seventy nine thousand one hundred and fifty seven seventy nine thousand one hundred and fifty eight seventy nine thousand one hun

In [4]:
# URL containing the JSON data
url = 'https://huggingface.co/datasets/ChavyvAkvar/synthetic-number/resolve/main/corpus_texts_val.json?download=true'

# Send a GET request to the URL
response = requests.get(url)

texts_val = []

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON data
    texts_val = response.json()

    # Check if the data is a list
    if isinstance(texts_val, list):
        # Now `data` is a Python list (array)
        print(texts_val[5])
    else:
        print("The data is not an array.")
else:
    print(f"Failed to retrieve data: {response.status_code}")

print(len(texts_val))

fifty eight thousand two hundred and eight fifty eight thousand two hundred and nine fifty eight thousand two hundred and ten fifty eight thousand two hundred and eleven fifty eight thousand two hundred and twelve fifty eight thousand two hundred and thirteen fifty eight thousand two hundred and fourteen fifty eight thousand two hundred and fifteen fifty eight thousand two hundred and sixteen fifty eight thousand two hundred and seventeen fifty eight thousand two hundred and eighteen fifty eight thousand two hundred and nineteen fifty eight thousand two hundred and twenty fifty eight thousand two hundred and twenty one fifty eight thousand two hundred and twenty two fifty eight thousand two hundred and twenty three fifty eight thousand two hundred and twenty four fifty eight thousand two hundred and twenty five fifty eight thousand two hundred and twenty six fifty eight thousand two hundred and twenty seven fifty eight thousand two hundred and twenty eight fifty eight thousand two hund

In [5]:
import torch
import torch.nn as nn
import math

def create_1d_rope_embeddings(seq_len, dim, device='cpu'):
    # Ensure the dimension is even
    assert dim % 2 == 0, "Dimension must be even for 1D RoPE"

    # Create position indices
    inv_freq = 1.0 / (10000 ** (torch.arange(0, dim, 2).float() / dim))
    inv_freq = inv_freq.to(device)

    # Create position embeddings
    sinusoid_inp = torch.einsum("i,j->ij", torch.arange(seq_len, dtype=torch.float, device=device), inv_freq)

    # Create the RoPE embeddings
    rope_sin = torch.sin(sinusoid_inp).unsqueeze(1).permute(1, 0, 2)
    rope_cos = torch.cos(sinusoid_inp).unsqueeze(1).permute(1, 0, 2)

    return rope_sin, rope_cos

def apply_1d_rope(token_embeddings, rope_sin, rope_cos):
    # Ensure the token embeddings dimension is even
    assert token_embeddings.shape[-1] % 2 == 0, "Token embeddings dimension must be even for 1D RoPE"

    # Split the token embeddings into even and odd dimensions
    token_embeddings_even = token_embeddings[..., 0::2]
    token_embeddings_odd = token_embeddings[..., 1::2]

    # Apply the RoPE embeddings
    token_embeddings_even_rotated = token_embeddings_even * rope_cos - token_embeddings_odd * rope_sin
    token_embeddings_odd_rotated = token_embeddings_even * rope_sin + token_embeddings_odd * rope_cos

    # Concatenate the rotated embeddings
    token_embeddings_rotated = torch.cat((token_embeddings_even_rotated, token_embeddings_odd_rotated), dim=-1)
    return token_embeddings_rotated

In [6]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

# Train a BPE tokenizer
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
tokenizer.pre_tokenizer = Whitespace()
trainer = BpeTrainer(vocab_size=500, special_tokens=["[UNK]", "[PAD]", "[EOS]"])
tokenizer.train_from_iterator(texts, trainer)

# Save and load the tokenizer
tokenizer.save("bpe_tokenizer.json")
tokenizer = Tokenizer.from_file("bpe_tokenizer.json")

In [7]:
import torch

class RMSNorm(torch.nn.Module):
    def __init__(self, dim, eps = 1e-6):
        super().__init__()
        self.weight = torch.nn.Parameter(torch.ones(dim))
        self.eps = eps

    def forward(self, x):
        dtype = x.dtype
        x = x.float()
        norm = torch.mean(x * x, dim=-1, keepdim=True)
        xnorm = x * torch.rsqrt(norm + self.eps)
        xnorm = xnorm.to(dtype=dtype)
        return xnorm * self.weight

def justnorm(self, x):
        #return F.normalize(x, p=2, dim=-1)
        res = x / x.norm(p=2, dim=-1, keepdim=True)
        return res

In [8]:
import torch
from torch import nn

class SwiGLU(nn.Module):
    def __init__(self, dimension):
        super().__init__()
        self.linear_1 = nn.Linear(dimension,dimension)
        self.linear_2 = nn.Linear(dimension,dimension)

    def forward(self, x):
        output = self.linear_1(x)
        swish = output * torch.sigmoid(output)
        swiglu = swish * self.linear_2(x)

        return swiglu

In [22]:
import torch.nn.functional as F
from fastfeedforward import FFF
from liger_kernel.transformers import LigerRMSNorm, liger_rotary_pos_emb

def causal_mask(score, b, h, q_idx, kv_idx):
    return torch.where(q_idx >= kv_idx, score, -float("inf"))

class GroupedAttentionLayer(nn.Module):
    def __init__(self, dim, num_heads, num_groups, max_seq_len, device='cpu'):
        super(GroupedAttentionLayer, self).__init__()
        self.dim = dim
        self.num_heads = num_heads
        self.num_groups = num_groups
        self.head_dim = dim // num_heads
        self.max_seq_len = max_seq_len
        self.device = device

        # Linear projections for Q, K, V
        self.q_linear = nn.Linear(dim, dim, bias=True)
        self.k_linear = nn.Linear(dim, dim, bias=True)
        self.v_linear = nn.Linear(dim, dim, bias=True)
        self.out_linear = nn.Linear(dim, dim, bias=False)


    def forward(self, x):
        batch_size, seq_len, dim = x.shape

        # Precompute the RoPE embeddings
        rope_sin, rope_cos = create_1d_rope_embeddings(seq_len, self.head_dim, self.device)

        # Linear projections
        q = self.q_linear(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        k = self.k_linear(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        v = self.v_linear(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)

        # Apply RoPE to Q, K
        q, k = liger_rotary_pos_emb(q, k, rope_cos, rope_sin)

        # Group the queries, keys, and values
        q = q.reshape(batch_size, self.num_heads, self.num_groups, seq_len, self.head_dim // self.num_groups)
        k = k.reshape(batch_size, self.num_heads, self.num_groups, seq_len, self.head_dim // self.num_groups)
        v = v.reshape(batch_size, self.num_heads, self.num_groups, seq_len, self.head_dim // self.num_groups)

        # Compute attention using scaled_dot_product_attention
        attn_output = []
        for group in range(self.num_groups):
            attn_output.append(F.scaled_dot_product_attention(
                q[:, :, group], k[:, :, group], v[:, :, group], attn_mask=None, dropout_p=0.2, is_causal=True
            ))

        attn_output = torch.cat(attn_output, dim=2)

        # Reshape the output
        out = attn_output.transpose(1, 2).contiguous().view(batch_size, seq_len, dim)

        # Linear projection
        out = self.out_linear(out)

        return out

class TransformerLayer(nn.Module):
    def __init__(self, dim, num_heads, num_groups, max_seq_len, device='cpu'):
        super(TransformerLayer, self).__init__()
        self.attention = GroupedAttentionLayer(dim, num_heads, num_groups, max_seq_len, device)
        self.norm1 = LigerRMSNorm(dim)
        self.norm2 = LigerRMSNorm(dim)

        self.depth = 4 # your choice of the FFF depth
        self.leaf_width = math.ceil(dim*4 / 2**self.depth)
        self.region_leak = 0.3 # your choice of the region leak probability (0 - 0.5) to encourage generalisation in very deep FFFs

        self.ffn = FFF(
            input_width=dim,
            leaf_width=self.leaf_width,
            output_width=dim,
            depth=self.depth,
            activation=torch.nn.SiLU(),
            dropout=0.2,
            region_leak=self.region_leak
        )

    def forward(self, x):
        # Apply attention and add & norm
        x = self.norm1(x + self.attention(x))
        # Apply feed-forward network and add & norm
        x = self.norm2(x + self.ffn(x))
        return x

class TransformerModel(nn.Module):
    def __init__(self, num_layers, dim, num_heads, num_groups, max_seq_len, vocab_size, device='cpu'):
        super(TransformerModel, self).__init__()
        self.token_embedding = nn.Embedding(vocab_size, dim)
        self.layers = nn.ModuleList([
            TransformerLayer(dim, num_heads, num_groups, max_seq_len, device)
            for _ in range(num_layers)
        ])
        self.ln_f = RMSNorm(dim)
        self.head = nn.Linear(dim, vocab_size)

    def forward(self, x):
        # Token embeddings
        x = self.token_embedding(x)
        # Apply transformer layers
        for layer in self.layers:
            x = layer(x)
        # Final layer norm
        x = self.ln_f(x)
        # Output logits
        logits = self.head(x)
        return logits

In [10]:
import torch

# Tokenize and pad the training texts
encoded = tokenizer.encode_batch(texts)
sequences = [torch.tensor(enc.ids, dtype=torch.long) for enc in encoded]
pad_token_id = tokenizer.token_to_id("[PAD]")
seq_len = max(len(seq) for seq in sequences)
sequences_padded = [torch.cat([seq, torch.tensor([pad_token_id] * (seq_len - len(seq)), dtype=torch.long)]) for seq in sequences]
inputs = torch.stack(sequences_padded)
targets = torch.stack([torch.cat([seq[1:], torch.tensor([pad_token_id], dtype=torch.long)]) for seq in sequences_padded])
train_dataset = torch.utils.data.TensorDataset(inputs, targets)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True)

# Tokenize and pad the validation texts
encoded_val = tokenizer.encode_batch(texts_val)
sequences_val = [torch.tensor(enc.ids, dtype=torch.long) for enc in encoded_val]
seq_len_val = max(len(seq) for seq in sequences_val)
sequences_padded_val = [torch.cat([seq, torch.tensor([pad_token_id] * (seq_len_val - len(seq)), dtype=torch.long)]) for seq in sequences_val]
inputs_val = torch.stack(sequences_padded_val)
targets_val = torch.stack([torch.cat([seq[1:], torch.tensor([pad_token_id], dtype=torch.long)]) for seq in sequences_padded_val])
val_dataset = torch.utils.data.TensorDataset(inputs_val, targets_val)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=256, shuffle=False)

In [11]:
print(tokenizer.decode(inputs[5].squeeze().tolist(), skip_special_tokens=False))
print(tokenizer.decode(targets[5].squeeze().tolist(), skip_special_tokens=False))

seventy nine thousand one hundred and thirty nine seventy nine thousand one hundred and forty seventy nine thousand one hundred and forty one seventy nine thousand one hundred and forty two seventy nine thousand one hundred and forty three seventy nine thousand one hundred and forty four seventy nine thousand one hundred and forty five seventy nine thousand one hundred and forty six seventy nine thousand one hundred and forty seven seventy nine thousand one hundred and forty eight seventy nine thousand one hundred and forty nine seventy nine thousand one hundred and fifty seventy nine thousand one hundred and fifty one seventy nine thousand one hundred and fifty two seventy nine thousand one hundred and fifty three seventy nine thousand one hundred and fifty four seventy nine thousand one hundred and fifty five seventy nine thousand one hundred and fifty six seventy nine thousand one hundred and fifty seven seventy nine thousand one hundred and fifty eight seventy nine thousand one hun

In [12]:
from pytorch_optimizer import get_supported_optimizers

supported_optimizers = get_supported_optimizers()

print(supported_optimizers)

['a2grad', 'accsgd', 'adabelief', 'adabound', 'adadelta', 'adafactor', 'adahessian', 'adai', 'adalite', 'adalomo', 'adamax', 'adamg', 'adammini', 'adamod', 'adamp', 'adams', 'adamw', 'adan', 'adanorm', 'adapnm', 'adashift', 'adasmooth', 'ademamix', 'aggmo', 'aida', 'alig', 'amos', 'apollo', 'asgd', 'avagrad', 'bsam', 'came', 'dadaptadagrad', 'dadaptadam', 'dadaptadan', 'dadaptlion', 'dadaptsgd', 'diffgrad', 'fadam', 'fromage', 'galore', 'gravity', 'grokfastadamw', 'kate', 'lamb', 'lars', 'lion', 'lomo', 'madgrad', 'msvag', 'nero', 'novograd', 'padam', 'pid', 'pnm', 'prodigy', 'qhadam', 'qhm', 'radam', 'ranger', 'ranger21', 'scalableshampoo', 'schedulefreeadamw', 'schedulefreesgd', 'sgdp', 'sgdw', 'shampoo', 'signsgd', 'sm3', 'soap', 'sophiah', 'srmm', 'stableadamw', 'swats', 'tiger', 'yogi']


In [23]:
from pytorch_optimizer import create_optimizer
from liger_kernel.transformers import LigerCrossEntropyLoss

# Training Loop
model = TransformerModel(num_layers=4, dim=128, num_heads=8, num_groups=4, max_seq_len=seq_len, vocab_size=tokenizer.get_vocab_size(), device='cuda')
model.to('cuda')

optimizer = create_optimizer(
    model,
    'soap',
    lr=3e-3,
    weight_decay=0.01,
    betas=(.95, .95),
    precondition_frequency=10
)

# cross entropy layers from liger kernel
loss_fn = LigerCrossEntropyLoss(ignore_index=tokenizer.token_to_id("[PAD]"))

# Training loop with validation
for epoch in range(10):
    model.train()  # Set the model to training mode
    total_loss = 0
    for batch in train_dataloader:
        input_batch, target_batch = batch
        input_batch = input_batch.to('cuda', dtype=torch.long)
        target_batch = target_batch.to('cuda', dtype=torch.long)
        logits = model(input_batch)
        loss = loss_fn(logits.view(-1, tokenizer.get_vocab_size()), target_batch.view(-1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_train_loss = total_loss / len(train_dataloader)

    # Validation step
    model.eval()  # Set the model to evaluation mode
    total_val_loss = 0
    with torch.no_grad():  # Disable gradient calculation for validation
        for batch in val_dataloader:
            input_batch, target_batch = batch
            input_batch = input_batch.to('cuda', dtype=torch.long)
            target_batch = target_batch.to('cuda', dtype=torch.long)
            logits = model(input_batch)
            loss = loss_fn(logits.view(-1, tokenizer.get_vocab_size()), target_batch.view(-1))
            total_val_loss += loss.item()
    avg_val_loss = total_val_loss / len(val_dataloader)

    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss}, Val Loss: {avg_val_loss}")

Epoch 1, Train Loss: 2.6332762777805327, Val Loss: 1.7983605861663818
Epoch 2, Train Loss: 1.2653552249073983, Val Loss: 1.233238160610199
Epoch 3, Train Loss: 0.7214280687272548, Val Loss: 0.8758036196231842
Epoch 4, Train Loss: 0.3515706963837147, Val Loss: 0.6157311797142029
Epoch 5, Train Loss: 0.16570483837276698, Val Loss: 0.48903220891952515
Epoch 6, Train Loss: 0.1109884748235345, Val Loss: 0.4277656227350235
Epoch 7, Train Loss: 0.08708356898277998, Val Loss: 0.383316770195961
Epoch 8, Train Loss: 0.07043345663696528, Val Loss: 0.39620476961135864
Epoch 9, Train Loss: 0.05720775621011853, Val Loss: 0.3923240005970001
Epoch 10, Train Loss: 0.04752267682924867, Val Loss: 0.43493057787418365


In [24]:
# URL containing the JSON data
url = 'https://huggingface.co/datasets/ChavyvAkvar/synthetic-number/resolve/main/corpus_texts_test.json?download=true'

# Send a GET request to the URL
response = requests.get(url)

texts_test = []

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON data
    texts_test = response.json()

    # Check if the data is a list
    if isinstance(texts_test, list):
        # Now `data` is a Python list (array)
        print(texts_test[5])
    else:
        print("The data is not an array.")
else:
    print(f"Failed to retrieve data: {response.status_code}")

print(len(texts_test))

eighty two thousand six hundred and fifty eighty two thousand six hundred and fifty one eighty two thousand six hundred and fifty two
50


In [25]:
# URL containing the JSON data
url = 'https://huggingface.co/datasets/ChavyvAkvar/synthetic-number/resolve/main/corpus_texts_answer.json?download=true'

# Send a GET request to the URL
response = requests.get(url)

texts_answer = []

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON data
    texts_answer = response.json()

    # Check if the data is a list
    if isinstance(texts_answer, list):
        # Now `data` is a Python list (array)
        print(texts_answer[5])
    else:
        print("The data is not an array.")
else:
    print(f"Failed to retrieve data: {response.status_code}")

print(len(texts_answer))

eighty two thousand six hundred and fifty eighty two thousand six hundred and fifty one eighty two thousand six hundred and fifty two eighty two thousand six hundred and fifty three eighty two thousand six hundred and fifty four eighty two thousand six hundred and fifty five eighty two thousand six hundred and fifty six eighty two thousand six hundred and fifty seven eighty two thousand six hundred and fifty eight eighty two thousand six hundred and fifty nine eighty two thousand six hundred and sixty eighty two thousand six hundred and sixty one eighty two thousand six hundred and sixty two eighty two thousand six hundred and sixty three eighty two thousand six hundred and sixty four eighty two thousand six hundred and sixty five eighty two thousand six hundred and sixty six eighty two thousand six hundred and sixty seven eighty two thousand six hundred and sixty eight eighty two thousand six hundred and sixty nine eighty two thousand six hundred and seventy eighty two thousand six hu

In [26]:
import torch
import jiwer

def infer(model, tokenizer, texts_test, texts_answer, max_length=200, device='cuda'):
    generated_texts = []
    wer_scores = []
    reference_texts = []

    for input_text, reference_text in zip(texts_test, texts_answer):
        # Tokenize the input text
        encoded = tokenizer.encode(input_text)
        input_ids = torch.tensor(encoded.ids, dtype=torch.long).unsqueeze(0).to(device)

        # Generate text
        generated_ids = input_ids
        with torch.no_grad():
            for _ in range(max_length):
                # Pass the input through the model
                outputs = model(generated_ids)
                # Get the last token logits
                next_token_logits = outputs[:, -1, :]
                # Sample the next token
                next_token_id = torch.argmax(next_token_logits, dim=-1).unsqueeze(0)
                # Append the next token to the sequence
                generated_ids = torch.cat([generated_ids, next_token_id], dim=-1)
                # Stop if the EOS token is generated
                if next_token_id.item() == tokenizer.token_to_id("[EOS]"):
                    break

        # Decode the generated sequence
        generated_ids = generated_ids.squeeze().tolist()
        decoded_text = tokenizer.decode(generated_ids, skip_special_tokens=True)
        generated_texts.append(decoded_text)

        reference_text = tokenizer.encode(reference_text)
        reference_text = reference_text.ids[:len(generated_ids)]
        reference_text = tokenizer.decode(reference_text, skip_special_tokens=True)

        reference_texts.append(reference_text)

        # Calculate WER
        wer = jiwer.wer(reference_text, decoded_text)
        wer_scores.append(wer)

    return generated_texts, reference_texts, wer_scores

generated_texts, reference_texts, wer_scores = infer(model, tokenizer, texts_test, texts_answer, max_length=200)

for i, (input_text, generated_text, reference_text, wer) in enumerate(zip(texts_test, generated_texts, reference_texts, wer_scores)):
    print(f"Input text {i+1}: {input_text}")
    print(f"Generated text {i+1}: {generated_text}")
    print(f"Reference text {i+1}: {reference_text}")
    print(f"WER {i+1}: {wer}")
    print()

Input text 1: seventy eight thousand three hundred and fifty one seventy eight thousand three hundred and fifty two seventy eight thousand three hundred and fifty three
Generated text 1: seventy eight thousand three hundred and fifty one seventy eight thousand three hundred and fifty two seventy eight thousand three hundred and fifty three two thousand three hundred and fifty five sixty thousand and fifty six two forty three thousand and forty seven thousand and forty eight forty eight thousand and forty nine forty thousand and fifty one forty eight thousand and fifty two fifty thousand and fifty three forty eight thousand and fifty five fifty eight thousand and fifty six forty eight thousand and fifty seven fifty eight thousand and fifty eight sixty eight thousand and fifty nine fifty eight thousand and sixty thousand and sixty one eighteen thousand and sixty two fifty eight thousand and sixty eight sixty eight thousand and sixty two thousand and sixty three eleven fifty eight thousan

In [27]:
# Save the model weights as safetensors
def save_safetensors(model, path):
    state_dict = model.state_dict()
    # Convert state_dict to safetensors format
    save_file(state_dict, path)

# Call the function to save weights in safetensors format
save_safetensors(model, "llm-numbers/pytorch_model.safetensors")  # Replace with your model name

# Push the model and tokenizer to Hugging Face
api = HfApi()
api.upload_folder(
    folder_path="llm-numbers",
    repo_id="ChavyvAkvar/llm-numbers",  # Replace with your username and model name
    repo_type="model",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.safetensors:   0%|          | 0.00/3.32M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ChavyvAkvar/llm-numbers/commit/0d402f00f3f9a5dec6f19d1d0bda10c06ab2b901', commit_message='Upload folder using huggingface_hub', commit_description='', oid='0d402f00f3f9a5dec6f19d1d0bda10c06ab2b901', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ChavyvAkvar/llm-numbers', endpoint='https://huggingface.co', repo_type='model', repo_id='ChavyvAkvar/llm-numbers'), pr_revision=None, pr_num=None)